# Optimization

## Import Libraries

In [25]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
import pandas as pd

from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.core.problem import ElementwiseProblem

import src.optimization_utils as ou
import src.solutions as s


## Load Model

In [26]:
bundle = ou.load_model_bundle("../artifacts/mech_fail_model.joblib")

In [27]:
FEATURES = bundle["features"]
print(f"Training features ({len(FEATURES)}): {FEATURES}")


Training features (24): ['age', 'sex', 'PI_preop', 'PT_preop', 'LL_preop', 'SS_preop', 'T4PA_preop', 'L1PA_preop', 'SVA_preop', 'cobb_main_curve_preop', 'FC_preop', 'tscore_femneck_preop', 'HU_UIV_preop', 'HU_UIVplus1_preop', 'HU_UIVplus2_preop', 'num_levels', 'UIV_implant', 'num_fused_levels', 'ALIF', 'XLIF', 'TLIF', 'num_rods', 'num_screws', 'osteotomy']


In [28]:
UIV_CHOICES, xl, xu = ou.get_decision_config()

In [29]:
print("UIV_CHOICES:", UIV_CHOICES)
print("xl:", xl)
print("xu:", xu)

cols = ["uiv_code", "num_levels", "ALIF", "XLIF", "TLIF", "num_rods", "num_pelvic_screws", "osteotomy"]
print(pd.DataFrame([xl, xu], index=["xl","xu"], columns=cols))

UIV_CHOICES: ['hook', 'PS', 'FS']
xl: [0 1 0 0 0 1 1 0]
xu: [2 3 1 1 1 6 6 1]
    uiv_code  num_levels  ALIF  XLIF  TLIF  num_rods  num_pelvic_screws  \
xl         0           1     0     0     0         1                  1   
xu         2           3     1     1     1         6                  6   

    osteotomy  
xl          0  
xu          1  


## Test Patient w fixed parameters

In [30]:
patient_fixed = {
    "age": 65,
    "sex": "FEMALE",
    "bmi": 18.48,
    "C7CSVL_preop": -6.1,
    "SVA_preop": 40.7,
    "T4PA_preop": 7.4,
    "L1PA_preop": 9.1,
    "LL_preop": 51.8,
    "L4S1_preop": 32.5,
    "PT_preop": 14.1,
    "PI_preop": 45.8,
    "SS_preop": 31.7,
    "cobb_main_curve_preop": 60.2,
    "FC_preop": 10.6,
    "tscore_femneck_preop": -0.5,
    "HU_UIV_preop": 229,
    "HU_UIVplus1_preop": 245,
    "HU_UIVplus2_preop": 248,
}

ou.debug_candidate(
    x=np.array([0, 3, 1, 0, 0, 6, 4, 0]),
    patient_fixed=patient_fixed,
    bundle=bundle,
    uiv_choices=UIV_CHOICES,
)

{'x': [0, 3, 1, 0, 0, 6, 4, 0],
 'plan': {'UIV_implant': 'hook',
  'num_levels': 3,
  'ALIF': 1,
  'XLIF': 0,
  'TLIF': 0,
  'num_rods': 6,
  'num_pelvic_screws': 4,
  'osteotomy': 0},
 'mech_fail_prob': 0.06905152967374342,
 'fitness': 0.06905152967374342}

## Build optimization problem

In [31]:
def make_problem():
    def _evaluate(self, x, out, *args, **kwargs):
        f = ou.fitness_mech_fail_only(x, patient_fixed, bundle, UIV_CHOICES)
        out["F"] = np.array([f], dtype=float)

    ProblemType = type(
        "SpineProblem",
        (ElementwiseProblem,),
        {
            "__init__": lambda self: ElementwiseProblem.__init__(
                self,
                n_var=len(xl),
                n_obj=1,
                xl=xl,
                xu=xu,
                vtype=int,
            ),
            "_evaluate": _evaluate,
        },
    )
    return ProblemType()

problem = make_problem()

## Run GA and view results

In [32]:
algorithm = GA(
    pop_size=100,
    sampling=IntegerRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=3),
    eliminate_duplicates=True,
)

res = minimize(
    problem,
    algorithm,
    get_termination("n_gen", 15),
    seed=42,
    verbose=True,
    save_history=True
)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       97 |  0.5739980318 |  0.0426623250
     2 |      197 |  0.0769913100 |  0.0426623250
     3 |      297 |  0.0562201973 |  0.0426623250
     4 |      397 |  0.0475395358 |  0.0426623250
     5 |      497 |  0.0458238448 |  0.0426623250
     6 |      597 |  0.0443342238 |  0.0426623250
     7 |      697 |  0.0433734522 |  0.0426623250
     8 |      797 |  0.0427013184 |  0.0419200282
     9 |      897 |  0.0426474790 |  0.0419200282
    10 |      997 |  0.0426326331 |  0.0419200282
    11 |     1097 |  0.0426252101 |  0.0419200282
    12 |     1197 |  0.0426029412 |  0.0419200282
    13 |     1297 |  0.0425584034 |  0.0419200282
    14 |     1397 |  0.0424619048 |  0.0419200282
    15 |     1497 |  0.0422466388 |  0.0419200282


## Show best solution

In [33]:
best_x = np.asarray(res.X).astype(int)
best_plan = ou.decode_plan(best_x, UIV_CHOICES)

best_full = ou.build_full_input(patient_fixed, best_plan)
best_prob = ou.predict_mech_fail_prob(best_full, bundle)

print("Best mech_fail_prob:", best_prob)
best_plan

Best mech_fail_prob: 0.04192002818633948


{'UIV_implant': 'PS',
 'num_levels': 2,
 'ALIF': 0,
 'XLIF': 1,
 'TLIF': 0,
 'num_rods': 6,
 'num_pelvic_screws': 5,
 'osteotomy': 0}

## Getting multiple solutions

Currently forcing ALIF to 1 for all solutions because that is all we can evaluate with training data

**Get top 10 solutions from last generation**

In [ ]:
pop = res.algorithm.pop
X_pop = pop.get("X")
F_pop = pop.get("F").flatten()

order = np.argsort(F_pop)

TOP_SLICE = 1500
TOP_N = 10

rows = []
for idx in order[:TOP_SLICE]:
    x = np.asarray(X_pop[idx]).astype(int)
    plan = ou.decode_plan(x, UIV_CHOICES)
    rows.append({
        **plan,
        "fitness": float(F_pop[idx]),
    })

df = pd.DataFrame(rows)

df_unique = df.drop_duplicates(
    subset=["UIV_implant", "num_interbody_fusion_levels", "ALIF", "XLIF", "TLIF",
            "num_rods", "num_pelvic_screws", "osteotomy"],
    keep="first",
).reset_index(drop=True)

df_unique.head(TOP_N)

,UIV_implant,num_levels,ALIF,XLIF,TLIF,num_rods,num_pelvic_screws,osteotomy,fitness
0,PS,2,0,1,0,6,5,0,0.041920
1,PS,2,0,1,0,6,4,0,0.041920
2,PS,2,0,1,0,6,3,0,0.041920
3,PS,3,0,0,0,6,5,0,0.042662
4,PS,3,0,0,0,6,4,0,0.042662
5,PS,3,0,0,0,6,2,0,0.042662
6,PS,3,0,0,0,6,3,0,0.042662


In [35]:
df["UIV_implant"].value_counts()


UIV_implant
PS    100
Name: count, dtype: int64

In [36]:
solutions = s.get_diverse_solutions(
    res=res,
    top_n=5,
    top_per_gen=50,
    eps=0.1,
    bucket_cols=("UIV_implant", "XLIF", "TLIF"),
    n_per_bucket=1,
)

solutions


KeyError: Index(['num_fused_levels', 'num_screws'], dtype='object')